In [1]:
# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import requests
import scipy.stats as st
import math
import statistics


# Import API Keys
from api_keys import walk_key


In [2]:
file = "sales+school.csv"
df = pd.read_csv(file)
df = df.rename(columns = {"Total Sq. Ft.":"Total Sq Ft"})
df["dollar/sqft"] = (df["Sale Price"] / df["Total Sq Ft"]).astype(float)
df

,MLS ID,Sale Price,Close Date,Address,Zip Code,Total Sq Ft,Lot Size,Latitude,Longitude,High School,High School Rank,Middle School,Middle School Rank,Elementary School,Elementary School Rank,dollar/sqft
0,2001944,240000.0,2021-03-15,4917 BlueStem DR,80917,1170.0,711.0,38.8976243,-104.737074,Doherty High School,2.0,Russell Middle School,2.0,Carver Elementary School,1.0,205.128205
1,8359484,237500.0,2021-03-12,54 Villa DR,81001,2087.0,6509.0,38.8654252,-104.863362,Coronado High School,3.0,Holmes Middle School,3.0,Howbert Elementary School,3.0,113.799713
2,7312436,275000.0,2021-03-15,132 Hayden ST,80813,2503.0,9583.0,38.750711,-105.176383,Cripple Creek-Victor Junior-Senior High School,1.0,NaN,NaN,Cresson Elementary School,1.0,109.868158
3,1398523,566455.0,2021-03-18,6596 Wolf Gulch DR,80924,4714.0,9088.0,38.9603997,-104.703074,Liberty High School,4.0,Chinook Trail Middle School,NaN,Legacy Peak Elementary School,3.0,120.164404
4,4306524,410000.0,2021-03-12,6562 Tillamook DR,80925,1949.0,6600.0,38.8338816,-104.821363,Palmer High School,3.0,NaN,2.0,Coperni 2,NaN,210.364289
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4349,4628075,890000.0,2021-04-12,16 Polo DR,80906,3063.0,10890.0,38.7974702,-104.841016,Palmer High School,5.0,NaN,5.0,NaN,NaN,290.564806
4350,5664354,915000.0,2021-04-12,17925 Black Squirrel RD,80908,4808.0,625086.0,39.092056,-104.627332,Smoky Hill High School,NaN,NaN,NaN,Talbott Steam Innovation School,NaN,190.307820
4351,1719141,925000.0,2021-04-09,390 Paisley DR,80906,4822.0,13068.0,38.7435755,-104.835224,Mitchell High School,2.0,NaN,NaN,Stratmoor Hills Elementary School,5.0,191.829117
4352,3250460,990900.0,2021-04-15,4937 Rainbow Gulch TR,80924,6140.0,12120.0,38.9758208,-104.735368,Liberty High School,4.0,NaN,NaN,Pinello Elementary School,5.0,161.384365


In [6]:
# Making test call
url = f"https://api.walkscore.com/score?format=json&lat=38.8976243&lon=-104.737074&transit=1&bike=1&wsapikey={walk_key}"

response = requests.get(url)

response_json = response.json()
print(response.status_code)
print(json.dumps(response_json, indent=4))
print(response_json['walkscore'])

200
{
    "status": 1,
    "walkscore": 53,
    "description": "Somewhat Walkable",
    "updated": "2021-03-31 22:09:56.538152",
    "logo_url": "https://cdn.walk.sc/images/api-logo.png",
    "more_info_icon": "https://cdn.walk.sc/images/api-more-info.gif",
    "more_info_link": "https://www.redfin.com/how-walk-score-works",
    "ws_link": "https://www.walkscore.com/score/loc/lat=38.8976243/lng=-104.737074/?utm_source=robthompsonrealtor.com&utm_medium=ws_api&utm_campaign=ws_api",
    "help_link": "https://www.redfin.com/how-walk-score-works",
    "snapped_lat": 38.898,
    "snapped_lon": -104.7375,
    "transit": {
        "score": null,
        "description": null,
        "summary": null
    },
    "bike": {
        "score": 53,
        "description": "Bikeable"
    }
}
53


In [ ]:
# Doing actual API to retrieve walk score
walkscore = []
errors= []

count = 1
errorcount = 0
linebreak = "-----------------------------------------------------------------------------------------"

print("Beginning Walk Score Retrieval")
print(linebreak)

for row

7312436